In [ ]:
import os
import random
from PIL import Image


ICONS_DIR = os.path.expanduser('~/icon645/colored_icons_final')
OUTPUT_DIR = os.path.expanduser('~/icon645/icon_grids')

os.makedirs(OUTPUT_DIR, exist_ok=True)


icon_paths = []
for root, dirs, files in os.walk(ICONS_DIR):
    for fname in files:
        lower = fname.lower()
        if lower.endswith(('.png', '.jpg', '.jpeg')):
            icon_paths.append(os.path.join(root, fname))

if len(icon_paths) < 1:
    raise RuntimeError(f"Found no icons under {ICONS_DIR}.")


icon_pool = []
for path in icon_paths:
    icon_pool.append(path)
    icon_pool.append(path)



GRID_COLS = 3
GRID_ROWS = 3

CELL_SIZE = (128, 128)

ICON_MAX_SIZE = (80, 80)

CANVAS_SIZE = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)


for idx in range(1, 101):
    n_icons = random.randint(2, 9)

    chosen_icons = random.sample(icon_pool, k=n_icons)

    all_cells = list(range(GRID_COLS * GRID_ROWS))
    chosen_cells = random.sample(all_cells, k=n_icons)

    canvas = Image.new('RGBA', CANVAS_SIZE, (255, 255, 255, 0))

    for icon_path, cell_idx in zip(chosen_icons, chosen_cells):
        with Image.open(icon_path) as im:
            im = im.convert('RGBA')

            im.thumbnail(ICON_MAX_SIZE, Image.LANCZOS)

            row = cell_idx // GRID_COLS
            col = cell_idx % GRID_COLS
            x0 = col * CELL_SIZE[0]
            y0 = row * CELL_SIZE[1]

            paste_x = x0 + (CELL_SIZE[0] - im.width) // 2
            paste_y = y0 + (CELL_SIZE[1] - im.height) // 2

            canvas.paste(im, (paste_x, paste_y), im)

    out_name = f'grid_{idx:03d}.png'
    out_path = os.path.join(OUTPUT_DIR, out_name)
    canvas.save(out_path)
    print(f"Saved {out_name} containing {n_icons} icons "
          f"(allowing up to 2 repeats per icon).")

print("Done: 100 icon‐grid images created in", OUTPUT_DIR)


In [ ]:
pwd

In [ ]:
import os
import random
import math
from PIL import Image, ImageDraw

# ─── Configuration ───────────────────────────────────────────────────────────────

ICONS_DIR  = os.path.expanduser('~/icon645/colored_icons_final')
OUTPUT_DIR = os.path.expanduser('~/icon645/icon_grids_with_arrows')

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ─── Gather all icon file paths ──────────────────────────────────────────────────

icon_paths = []
for root, dirs, files in os.walk(ICONS_DIR):
    for fname in files:
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            icon_paths.append(os.path.join(root, fname))

if not icon_paths:
    raise RuntimeError(f"No icon images found under {ICONS_DIR}.")

# Allow up to two copies of each icon in the pool
icon_pool = icon_paths * 2

# ─── Grid parameters ────────────────────────────────────────────────────────────

GRID_COLS    = 3
GRID_ROWS    = 3
CELL_SIZE    = (128, 128)
ICON_MAX_SIZE = (80, 80)
CANVAS_SIZE  = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)

# ─── Helper to draw an arrow ─────────────────────────────────────────────────────

def draw_arrow(draw: ImageDraw.Draw, start: tuple[int,int], end: tuple[int,int], width=4):
    """
    Draw a straight arrow from start to end on the given Draw object.
    """
    # Draw main line
    draw.line([start, end], fill="black", width=width)
    
    # Compute arrowhead (two lines at 20° angle)
    angle = math.atan2(end[1] - start[1], end[0] - start[0])
    head_length = 15
    head_angle  = math.radians(20)
    
    # Left side of arrowhead
    left_angle = angle + math.pi - head_angle
    left_x = end[0] + head_length * math.cos(left_angle)
    left_y = end[1] + head_length * math.sin(left_angle)
    
    # Right side of arrowhead
    right_angle = angle + math.pi + head_angle
    right_x = end[0] + head_length * math.cos(right_angle)
    right_y = end[1] + head_length * math.sin(right_angle)
    
    # Draw arrowhead lines
    draw.line([ (left_x, left_y), end ], fill="black", width=width)
    draw.line([ (right_x, right_y), end ], fill="black", width=width)

# ─── Main loop: create 100 grids with arrows ─────────────────────────────────────

for idx in range(1, 101):
    # Choose how many icons to place (between 2 and 9)
    n_icons = random.randint(2, GRID_COLS * GRID_ROWS)
    chosen_icons = random.sample(icon_pool, k=n_icons)
    
    # Choose distinct grid cells for each icon
    all_cells     = list(range(GRID_COLS * GRID_ROWS))
    chosen_cells  = random.sample(all_cells, k=n_icons)
    
    # Create blank canvas
    canvas = Image.new('RGBA', CANVAS_SIZE, (255,255,255,0))
    draw   = ImageDraw.Draw(canvas)
    
    # Track center positions of each placed icon
    centers: list[tuple[str, tuple[int,int]]] = []
    
    # Paste each icon into its cell
    for icon_path, cell_idx in zip(chosen_icons, chosen_cells):
        with Image.open(icon_path) as im:
            im = im.convert('RGBA')
            im.thumbnail(ICON_MAX_SIZE, Image.LANCZOS)
            
            row = cell_idx // GRID_COLS
            col = cell_idx % GRID_COLS
            x0  = col * CELL_SIZE[0]
            y0  = row * CELL_SIZE[1]
            
            paste_x = x0 + (CELL_SIZE[0] - im.width) // 2
            paste_y = y0 + (CELL_SIZE[1] - im.height) // 2
            
            canvas.paste(im, (paste_x, paste_y), im)
            
            # Compute and store the icon's center point
            center_x = paste_x + im.width // 2
            center_y = paste_y + im.height // 2
            centers.append((icon_path, (center_x, center_y)))
    
    # Draw an arrow between two randomly chosen distinct entities
    if len(centers) >= 2:
        (path1, c1), (path2, c2) = random.sample(centers, 2)
        draw_arrow(draw, c1, c2)
    
    # Save the result
    out_name = f'grid_{idx:03d}.png'
    out_path = os.path.join(OUTPUT_DIR, out_name)
    canvas.save(out_path)
    print(f"Saved {out_name}: {n_icons} icons placed, arrow from\n"
          f"  {os.path.basename(path1)} at {c1}\n"
          f"  to {os.path.basename(path2)} at {c2}")

print("Done: 100 icon-grid images with arrows created in", OUTPUT_DIR)


In [ ]:
#important 1

import os
import random
import math
from PIL import Image, ImageDraw

# ─── Configuration ───────────────────────────────────────────────────────────────

ICONS_DIR  = os.path.expanduser('~/icon645/colored_icons_final')
OUTPUT_DIR = os.path.expanduser('~/icon645/icon_grids_with_arrows')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ─── Gather all icon file paths ──────────────────────────────────────────────────

icon_paths = []
for root, dirs, files in os.walk(ICONS_DIR):
    for fname in files:
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            icon_paths.append(os.path.join(root, fname))
if not icon_paths:
    raise RuntimeError(f"No icon images found under {ICONS_DIR}.")

# Allow up to two copies of each icon in the pool
icon_pool = icon_paths * 2

# ─── Grid parameters ────────────────────────────────────────────────────────────

GRID_COLS     = 3
GRID_ROWS     = 3
CELL_SIZE     = (128, 128)
ICON_MAX_SIZE = (80, 80)
CANVAS_SIZE   = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)

# ─── Helper to draw an arrow ─────────────────────────────────────────────────────

def draw_arrow(draw: ImageDraw.Draw, start: tuple[int,int], end: tuple[int,int], width=4):
    draw.line([start, end], fill="black", width=width)
    angle = math.atan2(end[1] - start[1], end[0] - start[0])
    head_length = 15
    head_angle  = math.radians(20)
    # left head
    la = angle + math.pi - head_angle
    lx = end[0] + head_length * math.cos(la)
    ly = end[1] + head_length * math.sin(la)
    # right head
    ra = angle + math.pi + head_angle
    rx = end[0] + head_length * math.cos(ra)
    ry = end[1] + head_length * math.sin(ra)
    draw.line([(lx, ly), end], fill="black", width=width)
    draw.line([(rx, ry), end], fill="black", width=width)

# ─── Main loop: create 100 grids with arrows ─────────────────────────────────────

for idx in range(1, 101):
    # 1) sample icons and cells
    n_icons      = random.randint(2, GRID_COLS * GRID_ROWS)
    chosen_icons = random.sample(icon_pool, k=n_icons)
    chosen_cells = random.sample(range(GRID_COLS * GRID_ROWS), k=n_icons)

    # 2) setup canvas
    canvas = Image.new('RGBA', CANVAS_SIZE, (255,255,255,0))
    draw   = ImageDraw.Draw(canvas)
    centers = []

    # 3) paste icons & record centers
    for icon_path, cell in zip(chosen_icons, chosen_cells):
        with Image.open(icon_path) as im:
            im = im.convert('RGBA')
            im.thumbnail(ICON_MAX_SIZE, Image.LANCZOS)
            row, col = divmod(cell, GRID_COLS)
            x0 = col * CELL_SIZE[0] + (CELL_SIZE[0] - im.width) // 2
            y0 = row * CELL_SIZE[1] + (CELL_SIZE[1] - im.height) // 2
            canvas.paste(im, (x0, y0), im)
            centers.append((os.path.basename(icon_path), (x0 + im.width//2, y0 + im.height//2)))

    # 4) draw one random arrow if possible
    arrow_info = ""
    if len(centers) >= 2:
        (n1, c1), (n2, c2) = random.sample(centers, 2)
        draw_arrow(draw, c1, c2)
        arrow_info = f", arrow from '{n1}' to '{n2}'"

    # 5) save and print summary
    out_name = f'grid_{idx:03d}.png'
    canvas.save(os.path.join(OUTPUT_DIR, out_name))
    print(f"Created {out_name}: placed {n_icons} icons{arrow_info}")

print("All 100 grids with arrows created in", OUTPUT_DIR)


In [ ]:
import os
import random
import math
import sys
from PIL import Image, ImageDraw

# ─── Configuration ───────────────────────────────────────────────────────────────

ICONS_DIR  = os.path.expanduser('~/icon645/colored_icons_final')
OUTPUT_DIR = os.path.expanduser('~/icon645/icon_grids_with_arrows')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ─── Gather all icon file paths ──────────────────────────────────────────────────

icon_paths = []
for root, dirs, files in os.walk(ICONS_DIR):
    for fname in files:
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            icon_paths.append(os.path.join(root, fname))
if not icon_paths:
    raise RuntimeError(f"No icon images found under {ICONS_DIR}.")

# Allow up to two copies of each icon in the pool
icon_pool = icon_paths * 2

# ─── Grid parameters ────────────────────────────────────────────────────────────

GRID_COLS     = 3
GRID_ROWS     = 3
CELL_SIZE     = (128, 128)
ICON_MAX_SIZE = (80, 80)
CANVAS_SIZE   = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)

# ─── Helper to draw an arrow ─────────────────────────────────────────────────────

def draw_arrow(draw: ImageDraw.Draw, start: tuple[int,int], end: tuple[int,int], width=4):
    # main line
    draw.line([start, end], fill="black", width=width)
    angle = math.atan2(end[1] - start[1], end[0] - start[0])
    head_length = 15
    head_angle  = math.radians(20)
    # left head
    la = angle + math.pi - head_angle
    lx = end[0] + head_length * math.cos(la)
    ly = end[1] + head_length * math.sin(la)
    # right head
    ra = angle + math.pi + head_angle
    rx = end[0] + head_length * math.cos(ra)
    ry = end[1] + head_length * math.sin(ra)
    draw.line([(lx, ly), end], fill="black", width=width)
    draw.line([(rx, ry), end], fill="black", width=width)

# ─── Main loop: create 100 grids with multiple arrows ────────────────────────────

if __name__ == "__main__":
    total = 100
    print("Saving grids into:", os.path.abspath(OUTPUT_DIR), flush=True)

    for idx in range(1, total + 1):
        # 1) pick number of icons and sample them
        n_icons      = random.randint(2, GRID_COLS * GRID_ROWS)
        chosen_icons = random.sample(icon_pool, k=n_icons)
        chosen_cells = random.sample(range(GRID_COLS * GRID_ROWS), k=n_icons)

        # 2) setup canvas
        canvas = Image.new('RGBA', CANVAS_SIZE, (255,255,255,0))
        draw   = ImageDraw.Draw(canvas)
        centers = []  # list of (name, (x,y))

        # 3) paste icons & record center positions
        for icon_path, cell in zip(chosen_icons, chosen_cells):
            with Image.open(icon_path) as im:
                im = im.convert('RGBA')
                im.thumbnail(ICON_MAX_SIZE, Image.LANCZOS)
                row, col = divmod(cell, GRID_COLS)
                x0 = col * CELL_SIZE[0] + (CELL_SIZE[0] - im.width) // 2
                y0 = row * CELL_SIZE[1] + (CELL_SIZE[1] - im.height) // 2
                canvas.paste(im, (x0, y0), im)
                cx = x0 + im.width // 2
                cy = y0 + im.height // 2
                centers.append((os.path.basename(icon_path), (cx, cy)))

        # 4) determine how many arrows: e.g. one arrow per two icons, at least 1
        num_arrows = random.randint(1, max(1, n_icons // 2))
        arrow_pairs = set()
        # sample unique pairs
        while len(arrow_pairs) < num_arrows and len(centers) >= 2:
            a, b = random.sample(range(len(centers)), 2)
            pair = tuple(sorted((a, b)))
            arrow_pairs.add(pair)

        # draw each arrow
        arrow_info = []
        for a, b in arrow_pairs:
            name1, c1 = centers[a]
            name2, c2 = centers[b]
            draw_arrow(draw, c1, c2)
            arrow_info.append(f"'{name1}'→'{name2}'")

        # 5) save and print summary
        out_name = f'grid_{idx:03d}.png'
        out_path = os.path.join(OUTPUT_DIR, out_name)
        canvas.save(out_path)
        arrows_txt = ", ".join(arrow_info)
        print(f"[{idx}/{total}] Created {out_name}: {n_icons} icons, arrows {arrows_txt}", flush=True)

    # final check
    num_files = len([f for f in os.listdir(OUTPUT_DIR) if f.lower().endswith('.png')])
    print(f"Finished: found {num_files} PNG files in {OUTPUT_DIR}", flush=True)


In [ ]:
import os
import random
from PIL import Image

# ------------------------
# Configuration
# ------------------------
ICONS_DIR = os.path.expanduser('~/icon645/colored_icons_final')
OUTPUT_DIR = os.path.expanduser('~/icon645/icons_rep_grid')

GRID_COLS = 3
GRID_ROWS = 3
CELL_SIZE = (128, 128)
ICON_MAX_SIZE = (80, 80)
CANVAS_SIZE = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)

# ------------------------
# Setup
# ------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)

icon_paths = []
print("Collecting icon paths...")
for root, _, files in os.walk(ICONS_DIR):
    for fname in files:
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            icon_paths.append(os.path.join(root, fname))
            if len(icon_paths) % 100 == 0:
                print(f"  Collected {len(icon_paths)} icons...", flush=True)

if not icon_paths:
    raise RuntimeError(f"No icons found in {ICONS_DIR}")

# ------------------------
# Generate icon grid images
# ------------------------
print("Generating icon grid images...\n")

for idx in range(1, 101):
    icon_path = random.choice(icon_paths)
    n_repeats = random.randint(2, 9)

    chosen_cells = random.sample(range(GRID_COLS * GRID_ROWS), k=n_repeats)
    canvas = Image.new('RGBA', CANVAS_SIZE, (255, 255, 255, 0))

    try:
        with Image.open(icon_path) as icon_im:
            icon_im = icon_im.convert('RGBA')
            icon_im.thumbnail(ICON_MAX_SIZE, Image.LANCZOS)

            for cell_idx in chosen_cells:
                row = cell_idx // GRID_COLS
                col = cell_idx % GRID_COLS
                x0 = col * CELL_SIZE[0]
                y0 = row * CELL_SIZE[1]

                paste_x = x0 + (CELL_SIZE[0] - icon_im.width) // 2
                paste_y = y0 + (CELL_SIZE[1] - icon_im.height) // 2

                canvas.paste(icon_im, (paste_x, paste_y), icon_im)

    except Exception as e:
        print(f" [!] Failed to load or process {icon_path}: {e}", flush=True)
        continue

    out_name = f"repgrid_{idx:03d}.png"
    out_path = os.path.join(OUTPUT_DIR, out_name)
    canvas.save(out_path)
    print(f"Saved {out_name} with {n_repeats}x: {os.path.basename(icon_path)}", flush=True)

print("\n✅ Done: 100 repeated-icon grid images saved in:", OUTPUT_DIR)


In [ ]:
import os
import random
from PIL import Image

# ------------------------
# Configuration
# ------------------------
ICONS_DIR  = os.path.expanduser('~/icon645/colored_icons_final')
OUTPUT_DIR = os.path.expanduser('~/icon645/icons_same_rep_grid')

GRID_COLS     = 3
GRID_ROWS     = 3
CELL_SIZE     = (128, 128)
ICON_MAX_SIZE = (80, 80)
CANVAS_SIZE   = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)

MAX_ICONS_TO_COLLECT = 10000

# ------------------------
# Setup
# ------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)

icon_paths = []
print("Collecting up to the first 10,000 icon paths...")
for root, _, files in os.walk(ICONS_DIR):
    for fname in files:
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            icon_paths.append(os.path.join(root, fname))
            if len(icon_paths) % 100 == 0:
                print(f"  Collected {len(icon_paths)} icons...", flush=True)
            if len(icon_paths) >= MAX_ICONS_TO_COLLECT:
                break
    if len(icon_paths) >= MAX_ICONS_TO_COLLECT:
        break

if not icon_paths:
    raise RuntimeError(f"No icons found in {ICONS_DIR}")

print(f"Finished collecting {len(icon_paths)} icons.\n")

# ------------------------
# Generate icon grid images
# ------------------------
print("Generating icon grid images...\n")

for idx in range(1, 101):
    # pick one icon at random (with replacement) and repeat it
    icon_path = random.choice(icon_paths)
    n_repeats = random.randint(2, 9)

    # choose which cells to fill
    chosen_cells = random.sample(range(GRID_COLS * GRID_ROWS), k=n_repeats)
    canvas = Image.new('RGBA', CANVAS_SIZE, (255, 255, 255, 0))

    try:
        with Image.open(icon_path) as icon_im:
            icon_im = icon_im.convert('RGBA')
            icon_im.thumbnail(ICON_MAX_SIZE, Image.LANCZOS)

            for cell_idx in chosen_cells:
                row, col = divmod(cell_idx, GRID_COLS)
                x0 = col * CELL_SIZE[0]
                y0 = row * CELL_SIZE[1]

                paste_x = x0 + (CELL_SIZE[0] - icon_im.width) // 2
                paste_y = y0 + (CELL_SIZE[1] - icon_im.height) // 2

                canvas.paste(icon_im, (paste_x, paste_y), icon_im)

    except Exception as e:
        print(f" [!] Failed to process {icon_path}: {e}", flush=True)
        continue

    out_name = f"samerepgrid_{idx:03d}.png"
    out_path = os.path.join(OUTPUT_DIR, out_name)
    canvas.save(out_path)
    print(f"Saved {out_name} with {n_repeats} copies of {os.path.basename(icon_path)}", flush=True)

print(f"\n✅ Done: 100 repeated-icon grid images saved in {OUTPUT_DIR}")


In [ ]:
import os
import random
from PIL import Image

# ------------------------
# Configuration
# ------------------------
ICONS_DIR  = os.path.expanduser('~/icon645/colored_icons_final')
OUTPUT_DIR = os.path.expanduser('~/icon645/icons_rep_grid')

GRID_COLS     = 3
GRID_ROWS     = 3
CELL_SIZE     = (128, 128)
ICON_MAX_SIZE = (80, 80)
CANVAS_SIZE   = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)

MAX_ICONS_TO_COLLECT = 10000
NUM_IMAGES           = 100

# ------------------------
# Setup
# ------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)

icon_paths = []
print("Collecting up to the first 10,000 icon paths...")
for root, _, files in os.walk(ICONS_DIR):
    for fname in files:
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            icon_paths.append(os.path.join(root, fname))
            if len(icon_paths) % 100 == 0:
                print(f"  Collected {len(icon_paths)} icons...", flush=True)
            if len(icon_paths) >= MAX_ICONS_TO_COLLECT:
                break
    if len(icon_paths) >= MAX_ICONS_TO_COLLECT:
        break

if not icon_paths:
    raise RuntimeError(f"No icons found in {ICONS_DIR}")

print(f"Finished collecting {len(icon_paths)} icons.\n")
print(f"Generating {NUM_IMAGES} icon grid images with at least 2 distinct icons each...\n")

# ------------------------
# Generate icon grid images
# ------------------------
for idx in range(1, NUM_IMAGES + 1):
    # total icons to place
    n_total = random.randint(2, GRID_COLS * GRID_ROWS)
    # ensure at least 2 distinct icons
    max_distinct = min(n_total, len(icon_paths))
    n_distinct = random.randint(2, max_distinct)

    # pick distinct icons
    distinct_icons = random.sample(icon_paths, k=n_distinct)
    # now build the list of n_total icons: one of each distinct, plus random extras
    icons_pool = distinct_icons.copy()
    if n_total > n_distinct:
        extras = random.choices(distinct_icons, k=n_total - n_distinct)
        icons_pool += extras

    # shuffle positions
    chosen_cells = random.sample(range(GRID_COLS * GRID_ROWS), k=n_total)
    canvas = Image.new('RGBA', CANVAS_SIZE, (255, 255, 255, 0))

    # place each icon
    for icon_path, cell_idx in zip(icons_pool, chosen_cells):
        try:
            with Image.open(icon_path) as icon_im:
                icon_im = icon_im.convert('RGBA')
                icon_im.thumbnail(ICON_MAX_SIZE, Image.LANCZOS)

                row, col = divmod(cell_idx, GRID_COLS)
                x0 = col * CELL_SIZE[0]
                y0 = row * CELL_SIZE[1]

                paste_x = x0 + (CELL_SIZE[0] - icon_im.width) // 2
                paste_y = y0 + (CELL_SIZE[1] - icon_im.height) // 2

                canvas.paste(icon_im, (paste_x, paste_y), icon_im)
        except Exception as e:
            print(f" [!] Failed to process {icon_path}: {e}", flush=True)
            continue

    out_name = f"repgrid_{idx:03d}.png"
    out_path = os.path.join(OUTPUT_DIR, out_name)
    canvas.save(out_path)
    print(f"Saved {out_name}: {n_total} icons ({n_distinct} distinct)", flush=True)

print(f"\n✅ Done: {NUM_IMAGES} images saved in {OUTPUT_DIR}")


In [ ]:
!pwd

In [ ]:
#!/usr/bin/env python3
import os
import random
from PIL import Image, ImageDraw


OUTPUT_DIR = os.path.expanduser('~/arrow_dataset_final')
os.makedirs(OUTPUT_DIR, exist_ok=True)



COLORS       = ['red', 'green', 'blue', 'black', 'orange']
DIRECTIONS   = ['up', 'down', 'left', 'right']
BIDIRECTIONS = ['horizontal', 'vertical']
LINE_TYPES   = ['horizontal', 'vertical']

GRID_COLS     = 3
GRID_ROWS     = 3
CELL_SIZE     = (128, 128)
ICON_MAX_SIZE = (80, 80)
CANVAS_SIZE   = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)
ICON_CENTER   = (ICON_MAX_SIZE[0] // 2, ICON_MAX_SIZE[1] // 2)
ARROW_LENGTH  = 30



def draw_single_arrow(direction: str, color: str) -> Image.Image:
    img = Image.new('RGBA', ICON_MAX_SIZE, (255, 255, 255, 0))
    draw = ImageDraw.Draw(img)
    cx, cy = ICON_CENTER
    head_len = int(ARROW_LENGTH * 0.3)

    if direction == 'right':
        end = (cx + ARROW_LENGTH, cy)
        pts = [(end[0], end[1]), (end[0] - head_len, end[1] - head_len), (end[0] - head_len, end[1] + head_len)]
    elif direction == 'left':
        end = (cx - ARROW_LENGTH, cy)
        pts = [(end[0], end[1]), (end[0] + head_len, end[1] - head_len), (end[0] + head_len, end[1] + head_len)]
    elif direction == 'up':
        end = (cx, cy - ARROW_LENGTH)
        pts = [(end[0], end[1]), (end[0] - head_len, end[1] + head_len), (end[0] + head_len, end[1] + head_len)]
    else:  # 'down'
        end = (cx, cy + ARROW_LENGTH)
        pts = [(end[0], end[1]), (end[0] - head_len, end[1] - head_len), (end[0] + head_len, end[1] - head_len)]

    draw.line([(cx, cy), end], fill=color, width=3)
    draw.polygon(pts, fill=color)
    return img

def draw_bidirectional(direction: str, color: str) -> Image.Image:
    img = Image.new('RGBA', ICON_MAX_SIZE, (255, 255, 255, 0))
    draw = ImageDraw.Draw(img)
    cx, cy = ICON_CENTER
    head_len = int(ARROW_LENGTH * 0.3)

    if direction == 'horizontal':
        start = (cx - ARROW_LENGTH, cy)
        end   = (cx + ARROW_LENGTH, cy)
        draw.line([start, end], fill=color, width=3)
        draw.polygon([(start[0], start[1]), (start[0] + head_len, start[1] - head_len), (start[0] + head_len, start[1] + head_len)], fill=color)
        draw.polygon([(end[0], end[1]), (end[0] - head_len, end[1] - head_len), (end[0] - head_len, end[1] + head_len)], fill=color)
    else:
        start = (cx, cy - ARROW_LENGTH)
        end   = (cx, cy + ARROW_LENGTH)
        draw.line([start, end], fill=color, width=3)
        draw.polygon([(start[0], start[1]), (start[0] - head_len, start[1] + head_len), (start[0] + head_len, start[1] + head_len)], fill=color)
        draw.polygon([(end[0], end[1]), (end[0] - head_len, end[1] - head_len), (end[0] + head_len, end[1] - head_len)], fill=color)
    return img

def draw_line(line_type: str, color: str) -> Image.Image:
    img = Image.new('RGBA', ICON_MAX_SIZE, (255, 255, 255, 0))
    draw = ImageDraw.Draw(img)
    cx, cy = ICON_CENTER

    if line_type == 'horizontal':
        start = (cx - ARROW_LENGTH, cy)
        end   = (cx + ARROW_LENGTH, cy)
    else:
        start = (cx, cy - ARROW_LENGTH)
        end   = (cx, cy + ARROW_LENGTH)

    draw.line([start, end], fill=color, width=3)
    return img



for idx in range(1, 101):
    canvas = Image.new('RGBA', CANVAS_SIZE, (255, 255, 255, 0))

    cell_idx = random.randint(0, 8)
    category = random.choice(['single', 'bi', 'line'])
    color    = random.choice(COLORS)

    if category == 'single':
        direction = random.choice(DIRECTIONS)
        icon = draw_single_arrow(direction, color)
        name_detail = direction
    elif category == 'bi':
        direction = random.choice(BIDIRECTIONS)
        icon = draw_bidirectional(direction, color)
        name_detail = direction
    else:
        line_type = random.choice(LINE_TYPES)
        icon = draw_line(line_type, color)
        name_detail = line_type

    row = cell_idx // GRID_COLS
    col = cell_idx % GRID_COLS
    x0 = col * CELL_SIZE[0]
    y0 = row * CELL_SIZE[1]
    paste_x = x0 + (CELL_SIZE[0] - icon.width) // 2
    paste_y = y0 + (CELL_SIZE[1] - icon.height) // 2

    canvas.paste(icon, (paste_x, paste_y), icon)

    # Save
    fname = f"{category}_{name_detail}_{color}_at_{cell_idx}_{idx:03d}.png"
    canvas.save(os.path.join(OUTPUT_DIR, fname))

print(f"Done: 100 grid-positioned arrow/line images saved in {OUTPUT_DIR}")


In [ ]:
#!/usr/bin/env python3
import os
import random
from PIL import Image, ImageDraw

OUTPUT_DIR = os.path.expanduser('~/arrow_dataset_final')
os.makedirs(OUTPUT_DIR, exist_ok=True)



COLORS       = ['red', 'green', 'blue', 'black', 'orange']
DIRECTIONS   = ['up', 'down', 'left', 'right']
BIDIRECTIONS = ['horizontal', 'vertical']
LINE_TYPES   = ['horizontal', 'vertical']

GRID_COLS     = 3
GRID_ROWS     = 3
CELL_SIZE     = (128, 128)
ICON_MAX_SIZE = (80, 80)                  
CANVAS_SIZE   = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)
ICON_CENTER   = (ICON_MAX_SIZE[0] // 2, ICON_MAX_SIZE[1] // 2)
ARROW_LENGTH  = ICON_MAX_SIZE[0] // 2      



def draw_single_arrow(direction: str, color: str) -> Image.Image:
    img  = Image.new('RGBA', ICON_MAX_SIZE, (255,255,255,0))
    draw = ImageDraw.Draw(img)
    cx, cy    = ICON_CENTER
    head_len  = int(ARROW_LENGTH * 0.3)
    # compute tip & head points
    if direction == 'right':
        tip = (cx + ARROW_LENGTH, cy)
        head = [
            tip,
            (tip[0] - head_len, tip[1] - head_len),
            (tip[0] - head_len, tip[1] + head_len),
        ]
    elif direction == 'left':
        tip = (cx - ARROW_LENGTH, cy)
        head = [
            tip,
            (tip[0] + head_len, tip[1] - head_len),
            (tip[0] + head_len, tip[1] + head_len),
        ]
    elif direction == 'up':
        tip = (cx, cy - ARROW_LENGTH)
        head = [
            tip,
            (tip[0] - head_len, tip[1] + head_len),
            (tip[0] + head_len, tip[1] + head_len),
        ]
    else:  # down
        tip = (cx, cy + ARROW_LENGTH)
        head = [
            tip,
            (tip[0] - head_len, tip[1] - head_len),
            (tip[0] + head_len, tip[1] - head_len),
        ]

    draw.line([ (cx,cy), tip ], fill=color, width=3)
    draw.polygon(head, fill=color)
    return img

def draw_bidirectional(direction: str, color: str) -> Image.Image:
    img  = Image.new('RGBA', ICON_MAX_SIZE, (255,255,255,0))
    draw = ImageDraw.Draw(img)
    cx, cy   = ICON_CENTER
    head_len = int(ARROW_LENGTH * 0.3)
    if direction == 'horizontal':
        start = (cx - ARROW_LENGTH, cy)
        end   = (cx + ARROW_LENGTH, cy)
        draw.line([start, end], fill=color, width=3)
        # left head
        draw.polygon([
            start,
            (start[0] + head_len, start[1] - head_len),
            (start[0] + head_len, start[1] + head_len),
        ], fill=color)
        # right head
        draw.polygon([
            end,
            (end[0] - head_len, end[1] - head_len),
            (end[0] - head_len, end[1] + head_len),
        ], fill=color)
    else:
        start = (cx, cy - ARROW_LENGTH)
        end   = (cx, cy + ARROW_LENGTH)
        draw.line([start, end], fill=color, width=3)
        # top head
        draw.polygon([
            start,
            (start[0] - head_len, start[1] + head_len),
            (start[0] + head_len, start[1] + head_len),
        ], fill=color)
        # bottom head
        draw.polygon([
            end,
            (end[0] - head_len, end[1] - head_len),
            (end[0] + head_len, end[1] - head_len),
        ], fill=color)
    return img

def draw_line(line_type: str, color: str) -> Image.Image:
    img  = Image.new('RGBA', ICON_MAX_SIZE, (255,255,255,0))
    draw = ImageDraw.Draw(img)
    cx, cy = ICON_CENTER
    if line_type == 'horizontal':
        start = (cx - ARROW_LENGTH, cy)
        end   = (cx + ARROW_LENGTH, cy)
    else:
        start = (cx, cy - ARROW_LENGTH)
        end   = (cx, cy + ARROW_LENGTH)
    draw.line([start, end], fill=color, width=3)
    return img


for idx in range(1, 101):
    canvas = Image.new('RGBA', CANVAS_SIZE, (255,255,255,0))

    for cell_idx in range(GRID_COLS * GRID_ROWS):
        category = random.choice(['single','bi','line'])
        color    = random.choice(COLORS)

        if category == 'single':
            direction = random.choice(DIRECTIONS)
            icon = draw_single_arrow(direction, color)
            detail = direction
        elif category == 'bi':
            direction = random.choice(BIDIRECTIONS)
            icon = draw_bidirectional(direction, color)
            detail = direction
        else:
            line_type = random.choice(LINE_TYPES)
            icon = draw_line(line_type, color)
            detail = line_type

        row, col = divmod(cell_idx, GRID_COLS)
        x0 = col * CELL_SIZE[0] + (CELL_SIZE[0] - ICON_MAX_SIZE[0]) // 2
        y0 = row * CELL_SIZE[1] + (CELL_SIZE[1] - ICON_MAX_SIZE[1]) // 2
        canvas.paste(icon, (x0, y0), icon)

    fname = f"grid_{idx:03d}.png"
    canvas.save(os.path.join(OUTPUT_DIR, fname))

print(f" Done: 100 full 3×3 arrow grids saved in {OUTPUT_DIR}")


In [ ]:
#!/usr/bin/env python3
import os
import random
from PIL import Image, ImageDraw

# ───────────────────────────────────────────────────────────────────────────────
# 1) Output directory for final images
OUTPUT_DIR = os.path.expanduser('~/arrow_dataset_only_one_final')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ───────────────────────────────────────────────────────────────────────────────
# 2) Parameters

COLORS       = ['red', 'green', 'blue', 'black', 'orange']
DIRECTIONS   = ['up', 'down', 'left', 'right']
BIDIRECTIONS = ['horizontal', 'vertical']
LINE_TYPES   = ['horizontal', 'vertical']

GRID_COLS     = 3
GRID_ROWS     = 3
CELL_SIZE     = (128, 128)
ICON_MAX_SIZE = (80, 80)                  # arrow canvas size = icon size
CANVAS_SIZE   = (CELL_SIZE[0] * GRID_COLS, CELL_SIZE[1] * GRID_ROWS)
ICON_CENTER   = (ICON_MAX_SIZE[0] // 2, ICON_MAX_SIZE[1] // 2)
ARROW_LENGTH  = ICON_MAX_SIZE[0] // 2      # make arrow reach cell edges nicely

# ───────────────────────────────────────────────────────────────────────────────
# 3) Drawing functions (always output ICON_MAX_SIZE canvas)

def draw_single_arrow(direction: str, color: str) -> Image.Image:
    img  = Image.new('RGBA', ICON_MAX_SIZE, (255,255,255,0))
    draw = ImageDraw.Draw(img)
    cx, cy    = ICON_CENTER
    head_len  = int(ARROW_LENGTH * 0.3)
    if direction == 'right':
        tip = (cx + ARROW_LENGTH, cy)
        head = [
            tip,
            (tip[0] - head_len, tip[1] - head_len),
            (tip[0] - head_len, tip[1] + head_len),
        ]
    elif direction == 'left':
        tip = (cx - ARROW_LENGTH, cy)
        head = [
            tip,
            (tip[0] + head_len, tip[1] - head_len),
            (tip[0] + head_len, tip[1] + head_len),
        ]
    elif direction == 'up':
        tip = (cx, cy - ARROW_LENGTH)
        head = [
            tip,
            (tip[0] - head_len, tip[1] + head_len),
            (tip[0] + head_len, tip[1] + head_len),
        ]
    else:  # down
        tip = (cx, cy + ARROW_LENGTH)
        head = [
            tip,
            (tip[0] - head_len, tip[1] - head_len),
            (tip[0] + head_len, tip[1] - head_len),
        ]

    draw.line([(cx, cy), tip], fill=color, width=3)
    draw.polygon(head, fill=color)
    return img

def draw_bidirectional(direction: str, color: str) -> Image.Image:
    img  = Image.new('RGBA', ICON_MAX_SIZE, (255,255,255,0))
    draw = ImageDraw.Draw(img)
    cx, cy   = ICON_CENTER
    head_len = int(ARROW_LENGTH * 0.3)
    if direction == 'horizontal':
        start = (cx - ARROW_LENGTH, cy)
        end   = (cx + ARROW_LENGTH, cy)
        draw.line([start, end], fill=color, width=3)
        draw.polygon([start,
                      (start[0] + head_len, start[1] - head_len),
                      (start[0] + head_len, start[1] + head_len)],
                     fill=color)
        draw.polygon([end,
                      (end[0] - head_len, end[1] - head_len),
                      (end[0] - head_len, end[1] + head_len)],
                     fill=color)
    else:
        start = (cx, cy - ARROW_LENGTH)
        end   = (cx, cy + ARROW_LENGTH)
        draw.line([start, end], fill=color, width=3)
        draw.polygon([start,
                      (start[0] - head_len, start[1] + head_len),
                      (start[0] + head_len, start[1] + head_len)],
                     fill=color)
        draw.polygon([end,
                      (end[0] - head_len, end[1] - head_len),
                      (end[0] + head_len, end[1] - head_len)],
                     fill=color)
    return img

def draw_line(line_type: str, color: str) -> Image.Image:
    img  = Image.new('RGBA', ICON_MAX_SIZE, (255,255,255,0))
    draw = ImageDraw.Draw(img)
    cx, cy = ICON_CENTER
    if line_type == 'horizontal':
        start = (cx - ARROW_LENGTH, cy)
        end   = (cx + ARROW_LENGTH, cy)
    else:
        start = (cx, cy - ARROW_LENGTH)
        end   = (cx, cy + ARROW_LENGTH)
    draw.line([start, end], fill=color, width=3)
    return img

# ───────────────────────────────────────────────────────────────────────────────
# 4) Generate 100 images with at most one graphic per image
for idx in range(1, 101):
    canvas = Image.new('RGBA', CANVAS_SIZE, (255,255,255,0))

    # pick one random cell, graphic type, and color
    cell_idx = random.randint(0, GRID_COLS * GRID_ROWS - 1)
    category = random.choice(['single', 'bi', 'line'])
    color    = random.choice(COLORS)

    if category == 'single':
        direction = random.choice(DIRECTIONS)
        icon = draw_single_arrow(direction, color)
        detail = direction
    elif category == 'bi':
        direction = random.choice(BIDIRECTIONS)
        icon = draw_bidirectional(direction, color)
        detail = direction
    else:
        line_type = random.choice(LINE_TYPES)
        icon = draw_line(line_type, color)
        detail = line_type

    row, col = divmod(cell_idx, GRID_COLS)
    x = col * CELL_SIZE[0] + (CELL_SIZE[0] - ICON_MAX_SIZE[0]) // 2
    y = row * CELL_SIZE[1] + (CELL_SIZE[1] - ICON_MAX_SIZE[1]) // 2
    canvas.paste(icon, (x, y), icon)

    # save
    fname = f"{category}_{detail}_{color}_at_{cell_idx}_{idx:03d}.png"
    canvas.save(os.path.join(OUTPUT_DIR, fname))

print(f" Done: 100 single-item 3×3 grids saved in {OUTPUT_DIR}")
